This notebook is written based on [this reference implementation](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/6%20-%20Transformers%20for%20Sentiment%20Analysis.ipynb).

Other refs for model:
* https://stackoverflow.com/questions/65205582/how-can-i-add-a-bi-lstm-layer-on-top-of-bert-model
* https://discuss.pytorch.org/t/how-to-connect-hook-two-or-even-more-models-together/21033
* https://pytorch.org/tutorials/beginner/transformer_tutorial.html
* https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

Other refs for torchtext:
* https://towardsdatascience.com/use-torchtext-to-load-nlp-datasets-part-i-5da6f1c89d84
* https://towardsdatascience.com/use-torchtext-to-load-nlp-datasets-part-ii-f146c8b9a496
* http://anie.me/On-Torchtext/

# Imports and setup

In [3]:
import pandas as pd
import numpy as np
import os
import random
# random.seed(1)
import re

# Data processing.
import constants # constants.py
import dataset # dataset.py
import torch

# Model.
import models # models.py
import torch.nn as nn

# Training.
import training # training.py
import utils # utils.py

# If you make a code change that doesn't get picked up by
# Jupyter notebook, try reloading like below:
# import imp
# imp.reload(training)

# Read the data
Skip this section if you've already ran the notebook once and have the csvs locally.

In [3]:
# data_df = dataset.read_multiple_datasets([1,2,3], 'Creativity_Combined', shuffle=True)

In [4]:
'''This cell is commented out because the csvs should already exist in the directory.
If you are running the notebook for the first time, run them to generate the csvs.'''
# split into train, test sets. (Train set will be further split into 
# train+validation sets, via k-fold CV.)
# train_df = data_df[:1000]
# test_df = data_df[1000:] # roughly 203 test examples set aside

# write them to CSV files
# train_df.to_csv('ktrain.csv', index=False, header=False)
# test_df.to_csv('ktest.csv', index=False, header=False)

## Preprocessing and transform into torchtext Dataset format.

From what I understand, some preprocessing is done when data.Field() is applied.

In [21]:
train_dataset, test_dataset = dataset.get_train_test_datasets()

In [22]:
# Transform train_dataset into an np array representation.
# This will be used for generating the K folds.
train_exs_arr = np.array(train_dataset.examples)

# Training pipeline begins here


In [36]:
import imp
imp.reload(training)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

params = {
    'dropout': 0.2,
    'batch_size': 8,
    'max_epochs': 10,
    'lr': 5e-05
}

valid_corrs = training.launch_experiment(1, train_exs_arr, constants.N_SPLITS, params, save_weights=True)
print('validation correlations: {}'.format(valid_corrs))

training on fold 0


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


updating saved weights of best model
Epoch: 00 | Epoch Time: 0m 48s
	 Train Loss: 9.087 | Train Corr: 0.21
	 Val. Loss: 3.780 |  Val. Corr: 0.53
Epoch: 01 | Epoch Time: 0m 46s
	 Train Loss: 2.663 | Train Corr: 0.66
	 Val. Loss: 4.343 |  Val. Corr: 0.59
updating saved weights of best model
Epoch: 02 | Epoch Time: 0m 45s
	 Train Loss: 1.801 | Train Corr: 0.78
	 Val. Loss: 2.882 |  Val. Corr: 0.61
Epoch: 03 | Epoch Time: 0m 47s
	 Train Loss: 1.354 | Train Corr: 0.84
	 Val. Loss: 3.128 |  Val. Corr: 0.62
updating saved weights of best model
Epoch: 04 | Epoch Time: 0m 45s
	 Train Loss: 1.021 | Train Corr: 0.89
	 Val. Loss: 2.840 |  Val. Corr: 0.60
updating saved weights of best model
Epoch: 05 | Epoch Time: 0m 47s
	 Train Loss: 0.720 | Train Corr: 0.92
	 Val. Loss: 2.806 |  Val. Corr: 0.61
Epoch: 06 | Epoch Time: 0m 48s
	 Train Loss: 0.645 | Train Corr: 0.93
	 Val. Loss: 2.831 |  Val. Corr: 0.60
Epoch: 07 | Epoch Time: 0m 47s
	 Train Loss: 0.497 | Train Corr: 0.95
	 Val. Loss: 2.912 |  Val.

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: 00 | Epoch Time: 0m 48s
	 Train Loss: 8.447 | Train Corr: 0.25
	 Val. Loss: 5.427 |  Val. Corr: 0.57
Epoch: 01 | Epoch Time: 0m 46s
	 Train Loss: 3.042 | Train Corr: 0.58
	 Val. Loss: 3.730 |  Val. Corr: 0.58


KeyboardInterrupt: 

# Test the trained model on held-out dataset.

In [37]:
# Get a test iterator
test_iterator = training.get_iterator(test_dataset, params['batch_size'], device)

In [38]:
# load the best model saved
bert = DistilBertModel.from_pretrained(constants.WEIGHTS_NAME)
model = models.BERTLinear(bert, constants.OUTPUT_DIM, params['dropout'])
model.load_state_dict(torch.load("1_best_valid_loss.pt"))
model.to(device)
model.eval()
# If you change the criterion, make sure it matches with the training criterion in training.py
criterion = nn.MSELoss(size_average=False)
criterion = criterion.to(device)
test_loss, test_corr = training.evaluate(model, test_iterator, criterion, debug=True)
print(test_loss)
print(test_corr)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


predictions: tensor([3.8724, 4.2202, 2.7298, 4.1145, 4.0667, 3.1396, 3.8593, 3.8277],
       device='cuda:0')
true labels: tensor([3.8000, 4.9000, 3.4000, 4.5500, 4.1250, 2.5500, 4.5750, 3.0250],
       device='cuda:0')
predictions: tensor([4.1844, 4.3211, 4.7150, 4.8129, 3.1591, 4.6238, 4.4547, 4.6718],
       device='cuda:0')
true labels: tensor([3.9750, 5.0250, 4.6500, 5.1500, 4.0500, 4.6000, 4.2000, 4.0250],
       device='cuda:0')
predictions: tensor([3.3214, 4.8004, 3.8511, 3.9426, 3.5041, 4.4030, 3.2383, 4.0763],
       device='cuda:0')
true labels: tensor([2.8500, 4.7000, 3.4750, 4.1000, 4.9000, 4.9000, 2.7500, 4.0500],
       device='cuda:0')
predictions: tensor([3.9827, 4.5726, 4.4157, 4.8547, 4.1401, 4.6356, 4.2468, 3.7557],
       device='cuda:0')
true labels: tensor([3.6000, 4.1000, 4.7250, 5.2750, 3.5000, 5.0500, 4.1250, 2.9250],
       device='cuda:0')
predictions: tensor([3.7424, 4.3414, 4.5670, 3.9255, 3.8293, 4.2529, 4.4343, 4.6474],
       device='cuda:0')
true label

# Misc other stuff

Link to the trainer class: https://huggingface.co/transformers/main_classes/trainer.html



Default training arguments: https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments

Batch size per device: 8

Epoch: 3



This should be the model I used to generate my initial results: https://huggingface.co/transformers/model_doc/distilbert.html#distilbertforsequenceclassification
"DistilBert Model transformer with a sequence classification/regression head on top (a linear layer on top of the pooled output) e.g. for GLUE tasks."